reference:
- https://github.paypal.com/guazhu/NotebooksSharing/blob/master/notebooks/Dataproc_pyspark.ipynbm

note:
- run in pp notebook with pyspark kernel
- not tested in cosmos notebook environemt

# create a cluster in cmd

## create cluster using a template

In [ ]:
%%configuration --project ccg24-hrzana-gds-focus --dataproc -fa https://artifactory.paypalcorp.com/artifactory/big-data-platform/misc/bsi/2.6.0-SNAPSHOT/notebook_create_cluster_template.json --args cluster_name=guxia-bsi,project_id=ccg24-hrzana-gds-focus,cluster_size=small
{
    "computeCommonConfig": {
        "metadata": {
            "PIP_PACKAGES": "pandas==0.23.0 scipy==1.1.0", 
            "CONDA_PACKAGES": "tensorflow"
        }
    }
}

Starting Spark application


## create cluster using manual config

In [1]:
%%configuration --project ccg24-hrzana-gds-focus --dataproc
{
    "name": "guxia-gcp-test2",
    "image": "1.4",
    "software": {
        "properties": {
            "dataproc:dataproc.conscrypt.provider.enable": "false"
        }
    },
    "lifecycleConfig": {
        "idleDeleteTtl": "1d0h0m",
        "maxTtl": "1d0h0m"
    },
    "computeCommonConfig": {
        "metadata": {
            "PIP_PACKAGES": "pandas==0.24.2 scipy==1.1.0",
            "CONDA_PACKAGES": "tensorflow"
        }
    },
    "master": {
        "type": "n1-standard-16"
    },
    "workers": {
        "disk": {
            "DiskType": "pd-standard",
            "numLocalSsds": 1,
            "sizeInGb": 256
        },
        "type": "n1-highmem-8",
        "numInstances": 8
    },
    "secondaryWorkers": {
        "type": "n1-highmem-8",
        "disk": {
            "DiskType": "pd-standard",
            "numLocalSsds": 0,
            "sizeInGb": 256
        }
    }
}

Creating cluster with configuration:
{
    "image": "1.4",
    "lifecycleConfig": {
        "maxTtl": "1d0h0m",
        "idleDeleteTtl": "1d0h0m"
    },
    "name": "guxia-gcp-test2",
    "secondaryWorkers": {
        "type": "n1-highmem-8",
        "disk": {
            "sizeInGb": 256,
            "DiskType": "pd-standard",
            "numLocalSsds": 0
        }
    },
    "software": {
        "properties": {
            "dataproc:dataproc.conscrypt.provider.enable": "false"
        }
    },
    "workers": {
        "type": "n1-highmem-8",
        "numInstances": 8,
        "disk": {
            "sizeInGb": 256,
            "DiskType": "pd-standard",
            "numLocalSsds": 1
        }
    },
    "master": {
        "type": "n1-standard-16"
    },
    "computeCommonConfig": {
        "metadata": {
            "PIP_PACKAGES": "pandas==0.24.2 scipy==1.1.0",
            "CONDA_PACKAGES": "tensorflow"
        }
    }
}
Creating cluster guxia-gcp-test2
Cluster guxia-gcp-test2 status

## connecting existing cluster

In [ ]:
%cluster --dataproc --name guxia-6 --project ccg24-hrzana-gds-focus

## add jar to excution path

In [3]:
%%configure -f
{
    "conf": {
        "spark.jars":"gs://spark-lib/bigquery/spark-bigquery-latest.jar",
        "spark.driver.extraClassPath":"spark-bigquery-latest.jar",
        "spark.executor.extraClassPath":"spark-bigquery-latest.jar"
    }
}

## get a spark session

In [4]:


spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1689258897166_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.

# spark common usage

In [9]:
# run spark sql

spark.sql("show tables from edw").show(10)

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|     edw| account_action_view|      false|
|     edw|       account_party|      false|
|     edw| account_party_daily|      false|
|     edw|account_party_inc...|      false|
|     edw|account_party_mon...|      false|
|     edw|account_partyincr...|      false|
|     edw|account_partysnp_...|      false|
|     edw|account_property_...|      false|
|     edw|account_property_...|      false|
|     edw|account_property_...|      false|
+--------+--------------------+-----------+
only showing top 10 rows

In [8]:
# run sql to query table

spark.sql("""
            select
                account_number,
                wtransaction_base_id,
                wtransaction_parent_id,
                flags1,date_event_recorded 
            from edw.idi_txn_event_sender_increment 
            where dt='2021-09-01' 
            limit 10""")\
        .show()

+-------------------+--------------------+----------------------+--------+-------------------+
|     account_number|wtransaction_base_id|wtransaction_parent_id|  flags1|date_event_recorded|
+-------------------+--------------------+----------------------+--------+-------------------+
|1778524597166961830|   20748132657939060|                     0|38141952|2021-08-31 19:30:00|
|1996916945488670630|   20748129476052161|     20745937334752552| 4325376|2021-08-31 19:30:01|
|1509486644028769886|   20747933563951222|                     0|37748736|2021-08-31 19:30:02|
|1454599185646625822|   20748132657941739|                     0| 4456448|2021-08-31 19:30:03|
|1771413506920420502|   20747933564032936|     20748132657051097|38141952|2021-08-31 19:30:05|
|2020741488575112110|   20748107094280069|                     0| 4587520|2021-08-31 19:30:06|
|1790208466044208718|   20748132658031956|     20748136819653854|37879808|2021-08-31 19:30:07|
|1978179424939842062|   20748132658039004|        

In [32]:
# run some hive query

df = spark.sql("""describe table extended
                edw.idi_txn_event_sender_increment
                """)

df.show(10)

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|                  id|decimal(18,0)|   null|
|      account_number|decimal(38,0)|   null|
|wtransaction_base_id|decimal(38,0)|   null|
|event_sequence_nu...|decimal(18,0)|   null|
|wtransaction_shar...|decimal(38,0)|   null|
|wtransaction_pare...|decimal(38,0)|   null|
|         activity_id|       string|   null|
|time_event_published|       bigint|   null|
| date_event_recorded|       string|   null|
|            event_dt|       string|   null|
+--------------------+-------------+-------+
only showing top 10 rows

In [31]:
for r in df.where("col_name = 'Location'").select('data_type').take(1):
    print(r)

Row(data_type='gs://pypl-bkt-prd-row-std-edw/sys/pp_dt/SD/mozart/idi_txn_event_sender/sequence/incremental')

# query BQ

In [14]:
# config spark

spark.conf.set("viewsEnabled","true")
spark.conf.set("materializationProject","pypl-bods")
spark.conf.set("materializationDataset","rsh_row_gds_focus")
spark.conf.set("temporaryGcsBucket", "pypl-bkt-rsh-row-std-gds-focus")

In [19]:
# query BQ uds

sql = """
select
    trans_id
    , sndr_id 
    , rcvr_id 
    , pmt_start_date
from pypl-edw.pp_risk_ops_views.unified_dataset_v4 
where
    pmt_start_date between '2023-07-01' and '2023-07-01'
limit 10
"""

df = spark.read.format("bigquery").option("query", sql).load()

df.show()

+--------------------+-------------------+-------------------+--------------+
|            trans_id|            sndr_id|            rcvr_id|pmt_start_date|
+--------------------+-------------------+-------------------+--------------+
|-183797408602.000...|1915844909242874861|1414419792023701308|    2023-07-01|
|-183797408991.000...|1264487720233957504|1487837148248857168|    2023-07-01|
|-183797408850.000...|2046163580981915878|1592617174854680508|    2023-07-01|
|-183797396659.000...|1306152733027564313|1944253031083811403|    2023-07-01|
|-183797407246.000...|2093441894510465126|1944253031083811403|    2023-07-01|
|-183797377128.000...|1954517115921914226|1944253031083811403|    2023-07-01|
|-183797406314.000...|1586208010482336692|1944253031083811403|    2023-07-01|
|-183797408862.000...|2021937567426120519|2121522853507690020|    2023-07-01|
|-183797408941.000...|1422677759774730357|2252292727476257096|    2023-07-01|
|-183797408588.000...|1792819530557327422|2260763317790745551|  

In [20]:
df.printSchema()

root
 |-- trans_id: bignumeric (nullable = true)
 |-- sndr_id: string (nullable = true)
 |-- rcvr_id: string (nullable = true)
 |-- pmt_start_date: date (nullable = true)

# load file

In [5]:
parquet_data_dir = 'gs://pypl-bkt-rsh-row-std-gds-focus/user/guxia/file_center/cbp_madmen/parquet_20230101_20230131'

In [6]:
df = spark.read.parquet(parquet_data_dir)

In [16]:
for c in df.schema[-10:]:
    print(c)

StructField(vm_var_sfv3_ip_num_dist_acct_add_device,StringType,true)
StructField(vt_amt_pct_growth_2m,StringType,true)
StructField(wasn_br_gm,StringType,true)
StructField(wasn_pmts_gm,StringType,true)
StructField(weekOfYear,StringType,true)
StructField(xb_amt_ind_rate,StringType,true)
StructField(zip_sf_gloss_rate_12m,StringType,true)
StructField(zip_urt_rate_12m,StringType,true)
StructField(zip_urt_rate_3m,StringType,true)
StructField(zip_users_12m,StringType,true)

In [ ]:
for c in df.columns[-10:]:
    print(c)